https://www.oreilly.com/catalog/errata.csp?isbn=0636920142874

In [3]:
import tensorflow as tf;
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt


print(f'tensorflow version: {tf.__version__}')
print(f'keras version: {keras.__version__}')
print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))

tensorflow version: 2.9.1
keras version: 2.9.0
Num GPUs Available:  1


---

In [ ]:
import numpy as np
def heaviside(z):
    return (z >= 0).astype(z.dtype)

def mlp_xor(x1, x2, activation=heaviside):
    return activation(-activation(x1 + x2 - 1.5) + activation(x1 + x2 - 0.5) - 0.5)

# activation functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def relu(z):
    return np.maximum(0, z)

def derivative(f, z, eps=0.000001):
    return (f(z + eps) - f(z - eps))/(2 * eps)

In [ ]:
np.set_printoptions(suppress=True, formatter={'float_kind':'{:f}'.format})

In [ ]:
z = np.array([-1, 0, 0.1, 0.5, 0.9, 1, 2])
print(z)
print(relu(z))
print(derivative(relu, z))

In [ ]:
eps=0.000001
print(relu(0 + eps))
print(relu(0 - eps))
print(relu(0 + eps) - relu(0 - eps))
print(2*eps)
print(1e-06 / (2*eps))

---

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
X_train_full.shape

In [ ]:
y_train_full.shape

In [ ]:
# category of clothing
y_train_full[0:10]

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [ ]:
len(X_test)

In [ ]:
X_test[0].reshape(-1, 1)[0:300]

In [ ]:
plt.imshow(X_train[0], cmap="binary")
plt.axis('off')
plt.show()

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
class_names[y_train[0]]

In [ ]:
n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_train[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
#save_fig('fashion_mnist_plot', tight_layout=False)
plt.show()

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))
model.summary()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
# note the 1719 shown during training is not training size it is number of batches
len(X_train) / 32

In [ ]:
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))

In [ ]:
history.history.keys()

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
# note the 313 shown during training is not training size it is number of batches
len(X_test) / 32

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

In [ ]:
plt.figure(figsize=(7.2, 2.4))
for index, image in enumerate(X_new):
    plt.subplot(1, 3, index + 1)
    plt.imshow(image, cmap="binary", interpolation="nearest")
    plt.axis('off')
    plt.title(class_names[y_test[index]], fontsize=12)
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

**Warning**: `model.predict_classes(X_new)` is deprecated. It is replaced with `np.argmax(model.predict(X_new), axis=-1)`.

https://stackoverflow.com/questions/68776790/model-predict-classes-is-deprecated-what-to-use-instead

In [ ]:
import numpy as np
np.argmax(model.predict(X_new), axis=-1)

---

# TensorBoard

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [ ]:
import os
import numpy as np
root_logdir = os.path.join(os.curdir, "tensorflow_logs")
print(root_logdir)

In [ ]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])    
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [ ]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
history = model.fit(
    X_train, y_train, epochs=30,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpoint_cb, tensorboard_cb]
)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir=.my_logs --port=6006

---

# Callbacks and Early Stopping

In [4]:
import numpy as np

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [6]:
y_test.mean()

2.063185914728682

In [7]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

2022-07-10 20:21:20.144477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-10 20:21:21.380420: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10399 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1


In [8]:
import os
import numpy as np
root_logdir = os.path.join(os.curdir, "tensorflow_logs")
print(root_logdir)

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
print(run_logdir)

./tensorflow_logs
./tensorflow_logs/run_2022_07_10-20_21_21


In [9]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

In [10]:
history = model.fit(
    X_train, y_train,
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb],
)
model = keras.models.load_model("my_keras_model.h5") # rollback to best model
mse_test = model.evaluate(X_test, y_test)

162/162 [==============================] - 2s 3ms/step - loss: 0.3393


In [11]:
mse_test

0.339325487613678

In [12]:
y_test_pred = model.predict(X_test)

162/162 [==============================] - 0s 2ms/step


In [13]:
from helpsk.sklearn_eval import RegressionEvaluator
results = RegressionEvaluator(actual_values=y_test, predicted_values=y_test_pred)

In [14]:
results.mean_squared_error

0.33932539744236373

In [15]:
results.all_metrics

{'Mean Absolute Error (MAE)': 0.412066138545698,
 'Root Mean Squared Error (RMSE)': 0.5825164353409814,
 'RMSE to Standard Deviation of Target': 0.5063985645889937,
 'R Squared': 0.7435604937822069,
 'Total Observations': 5160}

---

# Hyperparameter Tuning

In [17]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [18]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [22]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

/tmp/ipykernel_11/1709004121.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


In [23]:
# keras_reg.fit(X_train, y_train, epochs=100,
#               validation_data=(X_valid, y_valid),
#               callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [63]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [1, 2, 3],
    "n_neurons": np.arange(1, 100)               .tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2)      .rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(
    keras_reg, param_distribs, 
    scoring='neg_mean_squared_error',
    n_iter=30,
    cv=5,
    verbose=2
)
rnd_search_cv.fit(
    X_train, y_train,
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Epoch 1/100
291/291 [==============================] - 2s 5ms/step - loss: 0.6993 - val_loss: 0.4683
Epoch 2/100
291/291 [==============================] - 1s 5ms/step - loss: 0.4401 - val_loss: 9.9101
Epoch 3/100
291/291 [==============================] - 1s 5ms/step - loss: 0.4064 - val_loss: 24.3733
Epoch 4/100
291/291 [==============================] - 1s 5ms/step - loss: 0.5167 - val_loss: 2.4425
Epoch 5/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3947 - val_loss: 10.6780
Epoch 6/100
291/291 [==============================] - 1s 5ms/step - loss: 0.5892 - val_loss: 2.0233
Epoch 7/100
291/291 [==============================] - 1s 5ms/step - loss: nan - val_loss: nan
Epoch 8/100
291/291 [==============================] - 1s 5ms/step - loss: nan - val_loss: nan
Epoch 9/100
291/291 [==============================] - 1s 5ms/step - loss: nan - val_loss: nan
Epoch 10/100
291/291 [=======================

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 219, in __call__
    return self._score(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 267, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_regression.py", line 442, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_regression.py", line 102, in _check_reg_targets
    y_pred = check_array(y_pred

291/291 [==============================] - 2s 5ms/step - loss: 0.6242 - val_loss: 1.5512
Epoch 2/100
291/291 [==============================] - 1s 5ms/step - loss: 0.4655 - val_loss: 62.8190
Epoch 3/100
291/291 [==============================] - 1s 5ms/step - loss: 2.5659 - val_loss: 35.6378
Epoch 4/100
291/291 [==============================] - 1s 5ms/step - loss: 0.5035 - val_loss: 0.4192
Epoch 5/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3975 - val_loss: 0.3986
Epoch 6/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3787 - val_loss: 0.4628
Epoch 7/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3691 - val_loss: 0.3513
Epoch 8/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3557 - val_loss: 0.3649
Epoch 9/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3469 - val_loss: 0.3402
Epoch 10/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3456 - val_loss: 0.3942

291/291 [==============================] - 1s 5ms/step - loss: 0.3294 - val_loss: 0.4783
Epoch 10/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3219 - val_loss: 1.0654
Epoch 11/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3167 - val_loss: 0.7144
Epoch 12/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3124 - val_loss: 1.8332
Epoch 13/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3090 - val_loss: 0.9627
Epoch 14/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3068 - val_loss: 1.1654
Epoch 15/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3050 - val_loss: 1.5049
Epoch 16/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3001 - val_loss: 0.7151
Epoch 17/100
291/291 [==============================] - 1s 5ms/step - loss: 0.2982 - val_loss: 2.0744
Epoch 18/100
291/291 [==============================] - 1s 5ms/step - loss: 0.2967 - val_loss: 

Epoch 40/100
73/73 [==============================] - 0s 2ms/step
[CV] END learning_rate=0.02452523437833673, n_hidden=2, n_neurons=54; total time=  57.2s
Epoch 1/100
291/291 [==============================] - 2s 5ms/step - loss: 3.1730 - val_loss: 7.7522
Epoch 2/100
291/291 [==============================] - 1s 5ms/step - loss: 1.3448 - val_loss: 5.2584
Epoch 3/100
291/291 [==============================] - 1s 5ms/step - loss: 0.9033 - val_loss: 2.8057
Epoch 4/100
291/291 [==============================] - 1s 5ms/step - loss: 0.7710 - val_loss: 1.5313
Epoch 5/100
291/291 [==============================] - 1s 5ms/step - loss: 0.7133 - val_loss: 0.9595
Epoch 6/100
291/291 [==============================] - 1s 5ms/step - loss: 0.6814 - val_loss: 0.7274
Epoch 7/100
291/291 [==============================] - 1s 5ms/step - loss: 0.6596 - val_loss: 0.6297
Epoch 8/100
291/291 [==============================] - 1s 5ms/step - loss: 0.6423 - val_loss: 0.6018
Epoch 9/100
291/291 [================

...

291/291 [==============================] - 1s 5ms/step - loss: 0.2797 - val_loss: 0.2925
Epoch 50/100
291/291 [==============================] - 1s 5ms/step - loss: 0.2793 - val_loss: 0.3294
Epoch 51/100
291/291 [==============================] - 1s 5ms/step - loss: 0.2782 - val_loss: 0.2943
Epoch 52/100
291/291 [==============================] - 1s 5ms/step - loss: 0.2767 - val_loss: 0.3327
Epoch 53/100
291/291 [==============================] - 1s 5ms/step - loss: 0.2771 - val_loss: 0.3149
Epoch 54/100
291/291 [==============================] - 1s 5ms/step - loss: 0.2769 - val_loss: 0.3071
Epoch 55/100
291/291 [==============================] - 1s 5ms/step - loss: 0.2749 - val_loss: 0.3006
Epoch 56/100
291/291 [==============================] - 1s 5ms/step - loss: 0.2746 - val_loss: 0.3452
Epoch 57/100
73/73 [==============================] - 0s 1ms/step
[CV] END learning_rate=0.008561187226597896, n_hidden=2, n_neurons=77; total time= 1.4min
Epoch 1/100
291/291 [====================

291/291 [==============================] - 1s 5ms/step - loss: 0.3778 - val_loss: 7.4964
Epoch 6/100
291/291 [==============================] - 1s 5ms/step - loss: 0.4109 - val_loss: 2.0569
Epoch 7/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3723 - val_loss: 0.5283
Epoch 8/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3500 - val_loss: 0.3548
Epoch 9/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3437 - val_loss: 0.3395
Epoch 10/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3392 - val_loss: 0.3603
Epoch 11/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3364 - val_loss: 0.4060
Epoch 12/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3308 - val_loss: 0.3259
Epoch 13/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3290 - val_loss: 0.3132
Epoch 14/100
291/291 [==============================] - 1s 5ms/step - loss: 0.3264 - val_loss: 0.39

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_search.py:953: UserWarning: One or more of the test scores are non-finite: [        nan -0.4036152  -0.37604469 -0.38304242 -0.36247813 -0.42347135
 -0.387753   -0.46393676 -0.34434366 -0.39667199 -0.38270865 -0.38470452
 -0.3685717  -0.30198147         nan -0.35470018 -0.32867325 -0.3891701
 -0.35197974 -0.35454755 -0.35789468 -0.35900512 -0.35781837 -0.35673999
 -0.35726278 -0.37579968 -0.37005245 -0.3709629  -0.36846959 -0.33066225]
  warnings.warn(


363/363 [==============================] - 2s 5ms/step - loss: 0.6242 - val_loss: 0.9081
Epoch 2/100
363/363 [==============================] - 2s 5ms/step - loss: 0.4127 - val_loss: 0.6481
Epoch 3/100
363/363 [==============================] - 2s 5ms/step - loss: 0.3762 - val_loss: 3.4548
Epoch 4/100
363/363 [==============================] - 2s 5ms/step - loss: 0.3772 - val_loss: 0.9674
Epoch 5/100
363/363 [==============================] - 2s 5ms/step - loss: 0.3498 - val_loss: 1.0139
Epoch 6/100
363/363 [==============================] - 2s 5ms/step - loss: 0.3466 - val_loss: 1.1609
Epoch 7/100
363/363 [==============================] - 2s 5ms/step - loss: 0.3345 - val_loss: 1.8363
Epoch 8/100
363/363 [==============================] - 2s 5ms/step - loss: 0.3333 - val_loss: 1.2753
Epoch 9/100
363/363 [==============================] - 2s 5ms/step - loss: 0.3248 - val_loss: 0.8347
Epoch 10/100
363/363 [==============================] - 2s 5ms/step - loss: 0.3192 - val_loss: 0.3382
E

RandomizedSearchCV(cv=5,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f41d64defa0>,
                   n_iter=30,
                   param_distributions={'learning_rate': [0.0029283955322296214,
                                                          0.0005554470248256512,
                                                          0.017722503949215675,
                                                          0.005964565522549639,
                                                          0.0006371073690873093,
                                                          0.00035028432352085515,
                                                          0.0005745668313494166,
                                                          0.00495127644585145,
                                                          0.0006850365207028326,
                                                          0.0008...
                                                          0.00046101702694268493,
                                                          0.0033027015327106953,
                                                          0.014121789252460085,
                                                          0.013724811259976951,
                                                          0.024512463918193082,
                                                          0.007475950923960584,
                                                          0.0007810206444742218,
                                                          0.00032936836542520444,
                                                          0.002728681534669279, ...],
                                        'n_hidden': [1, 2, 3],
                                        'n_neurons': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, ...]},
                   scoring='neg_mean_squared_error', verbose=2)

In [64]:
print(f"Best Score: {rnd_search_cv.best_score_}")
print(f"Best Params: {rnd_search_cv.best_params_}")

from helpsk.sklearn_eval import MLExperimentResults
results = MLExperimentResults.from_sklearn_search_cv(
    searcher=rnd_search_cv,
    higher_score_is_better=True,
    description='rnd_search_cv',
    # parameter_name_mappings=css.get_search_space_mappings(),
)

Best Score: -0.301981466575584
Best Params: {'n_neurons': 73, 'n_hidden': 3, 'learning_rate': 0.011771115152415022}


In [65]:
results.to_formatted_dataframe()

neg_mean_squared_error Mean,neg_mean_squared_error 95CI.LO,neg_mean_squared_error 95CI.HI,n_neurons,n_hidden,learning_rate
-0.302,-0.330,-0.274,73,3,0.012
-0.329,-0.368,-0.290,67,3,0.020
-0.331,-0.400,-0.261,77,2,0.009
-0.344,-0.397,-0.292,55,2,0.008
-0.352,-0.419,-0.285,57,1,0.011
-0.355,-0.408,-0.301,95,1,0.007
-0.355,-0.408,-0.302,19,2,0.003
-0.357,-0.395,-0.319,66,3,0.001
-0.357,-0.410,-0.305,60,3,0.009
-0.358,-0.400,-0.316,33,1,0.025


In [44]:
assert not np.isnan(X_train).any()

In [46]:
# from skopt import BayesSearchCV
# from skopt.space import Categorical, Real, Integer
# from sklearn.model_selection import RepeatedKFold

# n_splits = 5
# n_repeats = 2
# random_state = 42
# iterations = 30
# search_space = {
#     "n_hidden": Integer(0, 3),
#     "n_neurons": Real(1, 100),
#     "learning_rate": Real(3e-4, 3e-2),
# }
# bayes_search = BayesSearchCV(
#     estimator=keras_reg,
#     search_spaces=search_space,
#     cv=RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state),
#     scoring='neg_mean_squared_error',
#     n_iter=30,
#     refit=True,
#     return_train_score=False,
#     # n_jobs=-1,  CUDA_ERROR_OUT_OF_MEMORY: out of memory
#     verbose=1,
#     random_state=random_state,
# )
# bayes_search.fit(
#     X_train, y_train, epochs=100,
#     validation_data=(X_valid, y_valid),
#     callbacks=[keras.callbacks.EarlyStopping(patience=10)]
# )
# logging.info(f"Best Score: {bayes_search.best_score_}")
# logging.info(f"Best Params: {bayes_search.best_params_}")

# results = MLExperimentResults.from_sklearn_search_cv(
#     searcher=bayes_search,
#     higher_score_is_better=True,
#     description='BayesSearchCV',
#     parameter_name_mappings=css.get_search_space_mappings(),
# )